In [1]:
import numpy as np
import tensorflow as tf
import os

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

data_dir = os.path.join(os.getcwd(), 'weather-dataset')
class_names = os.listdir(data_dir)

batch_size = 32
img_height = 180
img_width = 180
num_classes = len(class_names)

class_names

['cloudy',
 'dew',
 'fogsmog',
 'frost',
 'glaze',
 'hail',
 'lightning',
 'rain',
 'rainbow',
 'rime',
 'sandstorm',
 'shine',
 'snow',
 'sunrise']

In [2]:
train_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_data = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_size = int(len(val_data) / 2)
test_data = val_data.take(test_size)
val_data = val_data.skip(test_size)

Found 7987 files belonging to 14 classes.
Using 6390 files for training.
Found 7987 files belonging to 14 classes.
Using 1597 files for validation.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)
test_data = test_data.cache().prefetch(buffer_size=AUTOTUNE)

In [4]:
from tensorflow.keras import layers, Sequential, optimizers

data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1)
])

In [5]:
def model_builder(hp):
  
  model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    data_augmentation,
    layers.Conv2D(hp.Int("conv1", min_value=32, max_value=96, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(hp.Int("conv2", min_value=64, max_value=128, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(hp.Int("conv3", min_value=64, max_value=128, step=32), 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(hp.Int("dense_units", min_value=128, max_value=768, step=128), activation='relu'),
    layers.Dense(num_classes)
  ])

  lr = hp.Choice("learning_rate", values=[1e-1, 1e-2, 1e-3, 1e-4])
  opt = optimizers.Adam(learning_rate=lr)

  model.compile(optimizer=opt,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [6]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
	monitor="val_loss",
	patience=2,
	restore_best_weights=True)

In [7]:
import keras_tuner as kt

epochs = 15

tuner = kt.Hyperband(
  model_builder,
  objective="val_accuracy",
  max_epochs=epochs,
  factor=3,
  seed=42,
  directory="./tuner-logs",
  project_name="hyperband"
)

In [8]:
tuner.search(
	x=train_data,
	validation_data=val_data,
	batch_size=batch_size,
	callbacks=[early_stop],
	epochs=epochs
)

bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of filters in conv_1 layer: {}".format(
	bestHP.get("conv_1")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(
	bestHP.get("conv_2")))
print("[INFO] optimal number of filters in conv_2 layer: {}".format(
	bestHP.get("conv_3")))
print("[INFO] optimal number of units in dense layer: {}".format(
	bestHP.get("dense_units")))
print("[INFO] optimal learning rate: {:.4f}".format(
	bestHP.get("learning_rate")))


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
64                |?                 |conv1
64                |?                 |conv2
96                |?                 |conv3
256               |?                 |dense_units
0.1               |?                 |learning_rate
2                 |?                 |tuner/epochs
0                 |?                 |tuner/initial_epoch
2                 |?                 |tuner/bracket
0                 |?                 |tuner/round

Epoch 1/2


In [ ]:
hypermodel = tuner.hypermodel.build(bestHP)
hypermodel.fit(train_data, validation_data=val_data, batch_size=batch_size, epochs=epochs, callbacks=[early_stop])

In [ ]:
hypermodel.save('./models/best-hypeparams-model/')